In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

/Users/gunnvantsaini/miniconda3/envs/tensorflow/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/gunnvantsaini/miniconda3/envs/tensorflow/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
facial=pd.read_csv("../sony/data/facial-keypoints-detection/training.csv")

In [3]:
facial.head(2)

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...


In [8]:
def flat_array(row):
    return np.array(row.split(" "),dtype="float")
facial['Image_new']=facial['Image'].map(flat_array)

In [10]:
facial['Image_new'].map(lambda x: len(x))

0       9216
1       9216
2       9216
3       9216
4       9216
5       9216
6       9216
7       9216
8       9216
9       9216
10      9216
11      9216
12      9216
13      9216
14      9216
15      9216
16      9216
17      9216
18      9216
19      9216
20      9216
21      9216
22      9216
23      9216
24      9216
25      9216
26      9216
27      9216
28      9216
29      9216
        ... 
7019    9216
7020    9216
7021    9216
7022    9216
7023    9216
7024    9216
7025    9216
7026    9216
7027    9216
7028    9216
7029    9216
7030    9216
7031    9216
7032    9216
7033    9216
7034    9216
7035    9216
7036    9216
7037    9216
7038    9216
7039    9216
7040    9216
7041    9216
7042    9216
7043    9216
7044    9216
7045    9216
7046    9216
7047    9216
7048    9216
Name: Image_new, Length: 7049, dtype: int64

In [6]:
facial[['left_eye_center_x','left_eye_center_y','right_eye_center_x','right_eye_center_y']].iloc[0].values

array([66.03356391, 39.00227368, 30.22700752, 36.4216782 ])

In [11]:
class Facial(Dataset):
    def __init__(self,table):
        table['Image']=table['Image'].map(lambda x: np.array(x.split(" "),dtype="float"))
        self.X=table['Image'].values
        self.y=table[['left_eye_center_x','left_eye_center_y','right_eye_center_x','right_eye_center_y']].values
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        keypoints=self.y[idx]
        kp={}
        kp['left_eye_center_x']=keypoints[0]
        kp['left_eye_center_y']=keypoints[1]
        kp['right_eye_center_x']=keypoints[2]
        kp['right_eye_center_y']=keypoints[3]
        img=np.resize(self.X[idx],(150,150))
        batch={'X':img,'y':kp}
        return batch

In [12]:
d=Facial(facial)

In [13]:
next(iter(d))


{'X': array([[238., 236., 237., ..., 147., 150., 149.],
        [152., 151., 149., ..., 240., 232., 201.],
        [182., 164., 121., ..., 152., 151., 149.],
        ...,
        [ 89.,  88.,  78., ..., 138., 131., 122.],
        [112., 113., 122., ...,  93., 105., 104.],
        [114., 123., 130., ..., 118., 119., 116.]]),
 'y': {'left_eye_center_x': 66.0335639098,
  'left_eye_center_y': 39.0022736842,
  'right_eye_center_x': 30.2270075188,
  'right_eye_center_y': 36.4216781955}}

In [14]:
D=DataLoader(d,batch_size=10)

In [15]:
next(iter(D))

{'X': tensor([[[238., 236., 237.,  ..., 147., 150., 149.],
          [152., 151., 149.,  ..., 240., 232., 201.],
          [182., 164., 121.,  ..., 152., 151., 149.],
          ...,
          [ 89.,  88.,  78.,  ..., 138., 131., 122.],
          [112., 113., 122.,  ...,  93., 105., 104.],
          [114., 123., 130.,  ..., 118., 119., 116.]],
 
         [[219., 215., 204.,  ..., 120., 116., 119.],
          [118., 114., 114.,  ..., 195., 196., 200.],
          [198., 205., 201.,  ..., 146., 155., 146.],
          ...,
          [ 60.,  84.,  91.,  ..., 194., 185., 183.],
          [180., 170., 158.,  ..., 110., 110., 116.],
          [116., 115., 115.,  ..., 115., 117., 122.]],
 
         [[144., 142., 159.,  ..., 107., 103., 109.],
          [110., 110., 108.,  ..., 103.,  85.,  71.],
          [ 57.,  48.,  58.,  ..., 110., 110., 105.],
          ...,
          [103., 140., 152.,  ..., 113., 114., 112.],
          [105.,  99.,  99.,  ..., 134., 131., 133.],
          [146., 156., 174

In [16]:
mnist=pd.read_csv("../sony/data/mnist_train.csv")

In [17]:
mnist.head(2)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
class Mnist(Dataset):
    def __init__(self,X,y):
        self.X=X
        self.y=y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        x=self.X[idx].reshape((1,28,28))
        y=self.y[idx]
        batch={"X":x,"y":y}
        return batch

In [65]:
X=mnist.drop('label',axis=1).values
y=mnist['label'].values

In [66]:
d=Mnist(X,y)

In [67]:
D=DataLoader(d,batch_size=32)

In [24]:
#### 
def O(I,K,P,S):
    return ((I-K+2*P)/S) +1

In [25]:
O(28,3,0,1) ### k=3,p=0,s=1

26.0

In [26]:
O(26,2,0,2) #### k=2,p=0,s=2

13.0

In [27]:
O(13,3,0,1) ### k=3,p=0,s=1

11.0

In [36]:
O(11,4,0,1) ### k=4,p=0,s=1

8.0

In [37]:
from torch import nn

In [40]:
### batch,C,H,W

In [54]:
class Lenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,padding=0,kernel_size=3,stride=1)
        self.pool1=nn.MaxPool2d(kernel_size=2,padding=0,stride=2)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,padding=0,stride=1)
        self.pool2=nn.MaxPool2d(kernel_size=4,padding=0,stride=1)
        self.linear1=nn.Linear(in_features=8*8*16,out_features=120)
        self.linear2=nn.Linear(in_features=120,out_features=84)
        self.linear3=nn.Linear(in_features=84,out_features=10)   
    def forward(self,X):
        X=self.conv1(X) ### batch_size,1, 28,28
        X=torch.tanh(X)
        X=self.pool1(X) ### batch_size,6,26,26 
        X=self.conv2(X) ### batch_size,16,13,13
        X=self.pool2(X) ###### batch_size,16,8,8
        X=X.view(-1,16*8*8) ### (batch_size,16*8*8)
        X=self.linear1(X)
        X=torch.tanh(X)
        X=self.linear2(X)
        X=torch.tanh(X)
        X=self.linear3(X)
        X=torch.softmax(X,axis=1)
        return X
        
    

In [55]:
mod=Lenet()

In [56]:
test_batch=torch.randn(3,1,28,28) ## b,c,h,w

In [58]:
mod(test_batch).size()

tensor([[0.1116, 0.1024, 0.1071, 0.0958, 0.0962, 0.1038, 0.0854, 0.1136, 0.0883,
         0.0959],
        [0.1132, 0.0997, 0.1093, 0.0964, 0.0951, 0.1024, 0.0870, 0.1143, 0.0886,
         0.0941],
        [0.1106, 0.1019, 0.1064, 0.0981, 0.0959, 0.1044, 0.0879, 0.1114, 0.0879,
         0.0956]], grad_fn=<SoftmaxBackward>)

In [68]:
next(iter(D))['X'].shape

torch.Size([32, 1, 28, 28])

In [69]:
import torch.optim as optim

In [70]:
mod=Lenet()
opt=optim.Adagrad(mod.parameters(),lr=0.001)
criteria=nn.CrossEntropyLoss()

In [73]:
for i in range(10):
    for batch in D:
        X=batch['X'].float()
        y=batch['y'].long()
        probs=mod(X)
        loss=criteria(probs,y)
        loss.backward()
        opt.step()
        opt.zero_grad()
        acc=(probs.argmax(axis=1)==y).float().mean().item()
    print(f"Loss: {round(loss.item(),3)}, Acc: {round(acc,3)}")

Loss: 1.65, Acc: 0.875
Loss: 1.568, Acc: 0.938
Loss: 1.532, Acc: 1.0
Loss: 1.514, Acc: 1.0
Loss: 1.503, Acc: 1.0
Loss: 1.495, Acc: 1.0
Loss: 1.489, Acc: 1.0
Loss: 1.483, Acc: 1.0
Loss: 1.48, Acc: 1.0
Loss: 1.477, Acc: 1.0


In [74]:
import json

In [75]:
con=open("/Users/gunnvantsaini/OneDrive/project_codes/content/dl_basics/deliverables_dl_basics/fashion_flat.json","r")
dat=con.read()
dat=json.loads(dat)
con.close()

In [76]:
dat.keys()

dict_keys(['y', 'x'])

In [78]:
len(dat['x'][0])

784

In [92]:
np.array(dat['x'][0],dtype="float").reshape((28,28)).shape

(28, 28)

In [85]:
class FashionData(Dataset):
    def __init__(self,data_dict):
        self.X=data_dict['x']
        self.y=data_dict['y']
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self,idx):
        x=self.X[idx]
        x=np.array(x,dtype='float').reshape((1,28,28))
        y=self.y[idx]
        batch={'X':x,"y":y}
        return batch

In [86]:
f_d=FashionData(dat)

In [87]:
next(iter(f_d))['X'].shape

(1, 28, 28)

In [88]:
F_D=DataLoader(f_d,batch_size=32)

In [89]:
mod=Lenet()
opt=optim.Adagrad(mod.parameters(),lr=0.001)
criteria=nn.CrossEntropyLoss()

In [90]:
for i in range(2):
    for batch in F_D:
        X=batch['X'].float()
        y=batch['y'].long()
        probs=mod(X)
        loss=criteria(probs,y)
        loss.backward()
        opt.step()
        opt.zero_grad()
        acc=(probs.argmax(axis=1)==y).float().mean().item()
        print(f"Loss: {round(loss.item(),3)}, Acc: {round(acc,3)}")

Loss: 2.303, Acc: 0.125
Loss: 2.296, Acc: 0.25
Loss: 2.299, Acc: 0.062
Loss: 2.304, Acc: 0.125
Loss: 2.29, Acc: 0.188
Loss: 2.271, Acc: 0.406
Loss: 2.27, Acc: 0.219
Loss: 2.272, Acc: 0.125
Loss: 2.271, Acc: 0.125
Loss: 2.251, Acc: 0.219
Loss: 2.26, Acc: 0.219
Loss: 2.243, Acc: 0.344
Loss: 2.239, Acc: 0.344
Loss: 2.244, Acc: 0.312
Loss: 2.23, Acc: 0.344
Loss: 2.255, Acc: 0.188
Loss: 2.23, Acc: 0.375
Loss: 2.246, Acc: 0.219
Loss: 2.201, Acc: 0.438
Loss: 2.231, Acc: 0.219
Loss: 2.231, Acc: 0.469
Loss: 2.204, Acc: 0.5
Loss: 2.179, Acc: 0.594
Loss: 2.196, Acc: 0.5
Loss: 2.17, Acc: 0.5
Loss: 2.183, Acc: 0.406
Loss: 2.191, Acc: 0.312
Loss: 2.201, Acc: 0.375
Loss: 2.173, Acc: 0.5
Loss: 2.165, Acc: 0.438
Loss: 2.194, Acc: 0.344
Loss: 2.157, Acc: 0.469
Loss: 2.181, Acc: 0.5
Loss: 2.175, Acc: 0.344
Loss: 2.192, Acc: 0.375
Loss: 2.192, Acc: 0.25
Loss: 2.155, Acc: 0.531
Loss: 2.13, Acc: 0.531
Loss: 2.126, Acc: 0.625
Loss: 2.183, Acc: 0.5
Loss: 2.153, Acc: 0.438
Loss: 2.141, Acc: 0.406
Loss: 2.185, 

Loss: 1.943, Acc: 0.625
Loss: 1.963, Acc: 0.531
Loss: 1.996, Acc: 0.531
Loss: 1.919, Acc: 0.594
Loss: 1.936, Acc: 0.562
Loss: 1.963, Acc: 0.594
Loss: 2.076, Acc: 0.406
Loss: 1.944, Acc: 0.562
Loss: 1.889, Acc: 0.688
Loss: 1.987, Acc: 0.531
Loss: 1.983, Acc: 0.438
Loss: 1.93, Acc: 0.625
Loss: 1.912, Acc: 0.594
Loss: 1.936, Acc: 0.594
Loss: 1.97, Acc: 0.594
Loss: 1.964, Acc: 0.562
Loss: 1.884, Acc: 0.656
Loss: 1.833, Acc: 0.75
Loss: 2.014, Acc: 0.531
Loss: 1.964, Acc: 0.531
Loss: 1.933, Acc: 0.594
Loss: 1.917, Acc: 0.594
Loss: 1.927, Acc: 0.625
Loss: 2.054, Acc: 0.406
Loss: 1.971, Acc: 0.562
Loss: 1.9, Acc: 0.656
Loss: 2.003, Acc: 0.531
Loss: 1.897, Acc: 0.625
Loss: 2.075, Acc: 0.406
Loss: 1.941, Acc: 0.625
Loss: 1.914, Acc: 0.531
Loss: 1.888, Acc: 0.656
Loss: 1.865, Acc: 0.719
Loss: 1.866, Acc: 0.719
Loss: 1.883, Acc: 0.719
Loss: 1.962, Acc: 0.656
Loss: 1.995, Acc: 0.562
Loss: 1.966, Acc: 0.562
Loss: 1.966, Acc: 0.531
Loss: 1.852, Acc: 0.688
Loss: 1.914, Acc: 0.562
Loss: 1.852, Acc: 0.6

Loss: 1.844, Acc: 0.719
Loss: 1.871, Acc: 0.688
Loss: 1.892, Acc: 0.594
Loss: 1.93, Acc: 0.625
Loss: 1.934, Acc: 0.562
Loss: 1.94, Acc: 0.594
Loss: 1.863, Acc: 0.656
Loss: 1.905, Acc: 0.625
Loss: 1.776, Acc: 0.719
Loss: 1.849, Acc: 0.688
Loss: 1.87, Acc: 0.656
Loss: 1.926, Acc: 0.594
Loss: 1.937, Acc: 0.625
Loss: 1.885, Acc: 0.656
Loss: 1.882, Acc: 0.656
Loss: 1.839, Acc: 0.688
Loss: 1.88, Acc: 0.656
Loss: 2.014, Acc: 0.469
Loss: 1.952, Acc: 0.562
Loss: 1.713, Acc: 0.844
Loss: 1.891, Acc: 0.594
Loss: 1.889, Acc: 0.594
Loss: 1.942, Acc: 0.562
Loss: 1.853, Acc: 0.688
Loss: 1.842, Acc: 0.719
Loss: 1.888, Acc: 0.688
Loss: 1.829, Acc: 0.625
Loss: 1.823, Acc: 0.719
Loss: 1.889, Acc: 0.594
Loss: 1.904, Acc: 0.531
Loss: 1.914, Acc: 0.531
Loss: 1.949, Acc: 0.5
Loss: 1.842, Acc: 0.719
Loss: 1.956, Acc: 0.562
Loss: 1.859, Acc: 0.656
Loss: 1.876, Acc: 0.594
Loss: 1.984, Acc: 0.5
Loss: 1.892, Acc: 0.594
Loss: 1.875, Acc: 0.719
Loss: 1.936, Acc: 0.562
Loss: 1.911, Acc: 0.625
Loss: 1.852, Acc: 0.719


Loss: 1.847, Acc: 0.656
Loss: 1.775, Acc: 0.719
Loss: 1.901, Acc: 0.625
Loss: 1.842, Acc: 0.688
Loss: 1.884, Acc: 0.562
Loss: 1.802, Acc: 0.719
Loss: 1.931, Acc: 0.562
Loss: 1.866, Acc: 0.594
Loss: 1.872, Acc: 0.656
Loss: 1.843, Acc: 0.75
Loss: 1.859, Acc: 0.719
Loss: 1.754, Acc: 0.812
Loss: 1.952, Acc: 0.5
Loss: 1.903, Acc: 0.656
Loss: 1.818, Acc: 0.688
Loss: 1.985, Acc: 0.469
Loss: 1.852, Acc: 0.625
Loss: 1.822, Acc: 0.688
Loss: 1.815, Acc: 0.719
Loss: 1.83, Acc: 0.688
Loss: 1.877, Acc: 0.594
Loss: 1.848, Acc: 0.625
Loss: 1.957, Acc: 0.531
Loss: 1.837, Acc: 0.625
Loss: 1.957, Acc: 0.625
Loss: 1.909, Acc: 0.531
Loss: 1.903, Acc: 0.656
Loss: 1.822, Acc: 0.688
Loss: 1.763, Acc: 0.812
Loss: 1.716, Acc: 0.875
Loss: 1.865, Acc: 0.656
Loss: 1.762, Acc: 0.812
Loss: 1.898, Acc: 0.562
Loss: 1.831, Acc: 0.625
Loss: 1.771, Acc: 0.781
Loss: 1.713, Acc: 0.812
Loss: 1.91, Acc: 0.562
Loss: 1.878, Acc: 0.625
Loss: 1.787, Acc: 0.781
Loss: 1.963, Acc: 0.562
Loss: 1.769, Acc: 0.719
Loss: 1.854, Acc: 0.6

Loss: 1.826, Acc: 0.688
Loss: 1.905, Acc: 0.594
Loss: 1.816, Acc: 0.719
Loss: 1.801, Acc: 0.75
Loss: 1.788, Acc: 0.688
Loss: 1.911, Acc: 0.5
Loss: 1.861, Acc: 0.594
Loss: 1.85, Acc: 0.656
Loss: 1.876, Acc: 0.656
Loss: 1.828, Acc: 0.625
Loss: 1.856, Acc: 0.594
Loss: 1.877, Acc: 0.688
Loss: 1.825, Acc: 0.688
Loss: 1.915, Acc: 0.562
Loss: 1.883, Acc: 0.656
Loss: 1.928, Acc: 0.5
Loss: 1.918, Acc: 0.562
Loss: 1.834, Acc: 0.656
Loss: 1.851, Acc: 0.688
Loss: 1.78, Acc: 0.656
Loss: 1.88, Acc: 0.594
Loss: 1.753, Acc: 0.781
Loss: 1.86, Acc: 0.656
Loss: 1.749, Acc: 0.781
Loss: 1.762, Acc: 0.812
Loss: 1.737, Acc: 0.812
Loss: 1.908, Acc: 0.625
Loss: 1.807, Acc: 0.656
Loss: 1.89, Acc: 0.594
Loss: 1.901, Acc: 0.562
Loss: 1.871, Acc: 0.625
Loss: 1.887, Acc: 0.594
Loss: 1.784, Acc: 0.656
Loss: 1.899, Acc: 0.625
Loss: 1.836, Acc: 0.656
Loss: 1.794, Acc: 0.688
Loss: 1.854, Acc: 0.625
Loss: 1.855, Acc: 0.688
Loss: 1.791, Acc: 0.688
Loss: 1.857, Acc: 0.75
Loss: 1.78, Acc: 0.688
Loss: 1.82, Acc: 0.688
Loss:

...

Loss: 1.65, Acc: 0.812
Loss: 1.754, Acc: 0.719
Loss: 1.876, Acc: 0.562
Loss: 1.734, Acc: 0.719
Loss: 1.879, Acc: 0.594
Loss: 1.665, Acc: 0.781
Loss: 1.645, Acc: 0.844
Loss: 1.628, Acc: 0.844
Loss: 1.683, Acc: 0.781
Loss: 1.621, Acc: 0.844
Loss: 1.725, Acc: 0.75
Loss: 1.773, Acc: 0.656
Loss: 1.765, Acc: 0.719
Loss: 1.811, Acc: 0.594
Loss: 1.767, Acc: 0.719
Loss: 1.738, Acc: 0.719
Loss: 1.681, Acc: 0.781
Loss: 1.804, Acc: 0.656
Loss: 1.682, Acc: 0.781
Loss: 1.729, Acc: 0.75
Loss: 1.911, Acc: 0.531
Loss: 1.674, Acc: 0.781
Loss: 1.814, Acc: 0.656
Loss: 1.701, Acc: 0.781
Loss: 1.793, Acc: 0.688
Loss: 1.72, Acc: 0.75
Loss: 1.764, Acc: 0.688
Loss: 1.698, Acc: 0.75
Loss: 1.696, Acc: 0.75
Loss: 1.766, Acc: 0.688
Loss: 1.622, Acc: 0.844
Loss: 1.795, Acc: 0.688
Loss: 1.633, Acc: 0.844
Loss: 1.65, Acc: 0.844
Loss: 1.8, Acc: 0.625
Loss: 1.645, Acc: 0.844
Loss: 1.726, Acc: 0.719
Loss: 1.724, Acc: 0.75
Loss: 1.841, Acc: 0.594
Loss: 1.675, Acc: 0.781
Loss: 1.706, Acc: 0.75
Loss: 1.708, Acc: 0.781
Loss

Loss: 1.801, Acc: 0.656
Loss: 1.781, Acc: 0.719
Loss: 1.652, Acc: 0.812
Loss: 1.734, Acc: 0.719
Loss: 1.8, Acc: 0.688
Loss: 1.744, Acc: 0.719
Loss: 1.756, Acc: 0.719
Loss: 1.777, Acc: 0.688
Loss: 1.779, Acc: 0.688
Loss: 1.727, Acc: 0.75
Loss: 1.615, Acc: 0.844
Loss: 1.759, Acc: 0.719
Loss: 1.589, Acc: 0.875
Loss: 1.656, Acc: 0.812
Loss: 1.751, Acc: 0.719
Loss: 1.755, Acc: 0.719
Loss: 1.914, Acc: 0.531
Loss: 1.748, Acc: 0.719
Loss: 1.909, Acc: 0.562
Loss: 1.757, Acc: 0.719
Loss: 1.729, Acc: 0.75
Loss: 1.78, Acc: 0.656
Loss: 1.67, Acc: 0.812
Loss: 1.625, Acc: 0.875
Loss: 1.869, Acc: 0.594
Loss: 1.728, Acc: 0.75
Loss: 1.776, Acc: 0.688
Loss: 1.637, Acc: 0.844
Loss: 1.768, Acc: 0.719
Loss: 1.648, Acc: 0.812
Loss: 1.85, Acc: 0.594
Loss: 1.677, Acc: 0.781
Loss: 1.714, Acc: 0.75
Loss: 1.815, Acc: 0.656
Loss: 1.644, Acc: 0.844
Loss: 1.717, Acc: 0.75
Loss: 1.74, Acc: 0.75
Loss: 1.709, Acc: 0.781
Loss: 1.751, Acc: 0.688
Loss: 1.757, Acc: 0.719
Loss: 1.732, Acc: 0.781
Loss: 1.764, Acc: 0.719
Loss

Loss: 1.664, Acc: 0.812
Loss: 1.558, Acc: 0.938
Loss: 1.843, Acc: 0.625
Loss: 1.741, Acc: 0.75
Loss: 1.771, Acc: 0.656
Loss: 1.697, Acc: 0.781
Loss: 1.811, Acc: 0.625
Loss: 1.658, Acc: 0.812
Loss: 1.629, Acc: 0.844
Loss: 1.748, Acc: 0.688
Loss: 1.737, Acc: 0.719
Loss: 1.735, Acc: 0.719
Loss: 1.846, Acc: 0.594
Loss: 1.76, Acc: 0.75
Loss: 1.899, Acc: 0.562
Loss: 1.689, Acc: 0.781
Loss: 1.684, Acc: 0.812
Loss: 1.647, Acc: 0.844
Loss: 1.769, Acc: 0.719
Loss: 1.746, Acc: 0.688
Loss: 1.732, Acc: 0.75
Loss: 1.748, Acc: 0.719
Loss: 1.731, Acc: 0.75
Loss: 1.688, Acc: 0.75
Loss: 1.728, Acc: 0.719
Loss: 1.843, Acc: 0.562
Loss: 1.697, Acc: 0.781
Loss: 1.69, Acc: 0.781
Loss: 1.692, Acc: 0.75
Loss: 1.652, Acc: 0.844
Loss: 1.63, Acc: 0.844
Loss: 1.728, Acc: 0.781
Loss: 1.817, Acc: 0.688
Loss: 1.654, Acc: 0.844
Loss: 1.717, Acc: 0.781
Loss: 1.805, Acc: 0.625
Loss: 1.632, Acc: 0.844
Loss: 1.617, Acc: 0.875
Loss: 1.771, Acc: 0.75
Loss: 1.757, Acc: 0.719
Loss: 1.756, Acc: 0.719
Loss: 1.729, Acc: 0.719
Lo

Loss: 1.757, Acc: 0.719
Loss: 1.753, Acc: 0.719
Loss: 1.719, Acc: 0.781
Loss: 1.729, Acc: 0.781
Loss: 1.695, Acc: 0.781
Loss: 1.724, Acc: 0.75
Loss: 1.691, Acc: 0.781
Loss: 1.701, Acc: 0.75
Loss: 1.765, Acc: 0.688
Loss: 1.704, Acc: 0.781
Loss: 1.776, Acc: 0.688
Loss: 1.742, Acc: 0.75
Loss: 1.72, Acc: 0.781
Loss: 1.661, Acc: 0.812
Loss: 1.769, Acc: 0.656
Loss: 1.801, Acc: 0.656
Loss: 1.626, Acc: 0.875
Loss: 1.729, Acc: 0.719
Loss: 1.645, Acc: 0.812
Loss: 1.656, Acc: 0.812
Loss: 1.781, Acc: 0.656
Loss: 1.762, Acc: 0.688
Loss: 1.819, Acc: 0.656
Loss: 1.682, Acc: 0.75
Loss: 1.727, Acc: 0.75
Loss: 1.626, Acc: 0.844
Loss: 1.622, Acc: 0.844
Loss: 1.75, Acc: 0.688
Loss: 1.76, Acc: 0.719
Loss: 1.743, Acc: 0.719
Loss: 1.639, Acc: 0.875
Loss: 1.655, Acc: 0.781
Loss: 1.759, Acc: 0.688
Loss: 1.706, Acc: 0.781
Loss: 1.774, Acc: 0.688
Loss: 1.92, Acc: 0.531
Loss: 1.827, Acc: 0.688
Loss: 1.769, Acc: 0.719
Loss: 1.761, Acc: 0.75
Loss: 1.692, Acc: 0.781
Loss: 1.74, Acc: 0.75
Loss: 1.624, Acc: 0.844
Loss

Loss: 1.756, Acc: 0.719
Loss: 1.672, Acc: 0.812
Loss: 1.754, Acc: 0.719
Loss: 1.712, Acc: 0.781
Loss: 1.839, Acc: 0.656
Loss: 1.781, Acc: 0.688
Loss: 1.877, Acc: 0.562
Loss: 1.712, Acc: 0.75
Loss: 1.689, Acc: 0.812
Loss: 1.71, Acc: 0.781
Loss: 1.743, Acc: 0.781
Loss: 1.759, Acc: 0.719
Loss: 1.884, Acc: 0.594
Loss: 1.619, Acc: 0.875
Loss: 1.773, Acc: 0.688
Loss: 1.825, Acc: 0.656
Loss: 1.824, Acc: 0.656
Loss: 1.73, Acc: 0.719
Loss: 1.863, Acc: 0.594
Loss: 1.712, Acc: 0.781
Loss: 1.737, Acc: 0.781
Loss: 1.843, Acc: 0.594
Loss: 1.718, Acc: 0.781
Loss: 1.666, Acc: 0.781
Loss: 1.741, Acc: 0.719
Loss: 1.712, Acc: 0.75
Loss: 1.694, Acc: 0.781
Loss: 1.705, Acc: 0.781
Loss: 1.76, Acc: 0.688
Loss: 1.775, Acc: 0.688
Loss: 1.72, Acc: 0.75
Loss: 1.625, Acc: 0.812
Loss: 1.72, Acc: 0.75
Loss: 1.833, Acc: 0.625
Loss: 1.636, Acc: 0.812
Loss: 1.803, Acc: 0.688
Loss: 1.751, Acc: 0.719
Loss: 1.791, Acc: 0.656
Loss: 1.651, Acc: 0.812
Loss: 1.658, Acc: 0.812
Loss: 1.701, Acc: 0.75
Loss: 1.793, Acc: 0.656
Lo

In [93]:
def fit(mod,opt,criteria, dataset,epochs):
    for i in range(epochs):
        for batch in dataset:
            X=batch['X'].float()
            y=batch['y'].long()
            probs=mod(X)
            loss=criteria(probs,y)
            loss.backward()
            opt.step()
            opt.zero_grad()
            acc=(probs.argmax(axis=1)==y).float().mean().item()
            print(f"Epoch : {i+1},Loss: {round(loss.item(),3)}, Acc: {round(acc,3)}")

In [94]:
mod=Lenet()
opt=optim.Adagrad(mod.parameters(),lr=0.001)
criteria=nn.CrossEntropyLoss()

In [95]:
fit(mod,opt,criteria,F_D,epochs=1) ## Trains

Epoch : 1,Loss: 2.303, Acc: 0.094
Epoch : 1,Loss: 2.299, Acc: 0.094
Epoch : 1,Loss: 2.302, Acc: 0.031
Epoch : 1,Loss: 2.312, Acc: 0.031
Epoch : 1,Loss: 2.301, Acc: 0.188
Epoch : 1,Loss: 2.284, Acc: 0.375
Epoch : 1,Loss: 2.281, Acc: 0.219
Epoch : 1,Loss: 2.285, Acc: 0.156
Epoch : 1,Loss: 2.276, Acc: 0.219
Epoch : 1,Loss: 2.272, Acc: 0.125
Epoch : 1,Loss: 2.268, Acc: 0.25
Epoch : 1,Loss: 2.264, Acc: 0.219
Epoch : 1,Loss: 2.259, Acc: 0.219
Epoch : 1,Loss: 2.258, Acc: 0.219
Epoch : 1,Loss: 2.244, Acc: 0.406
Epoch : 1,Loss: 2.27, Acc: 0.219
Epoch : 1,Loss: 2.238, Acc: 0.438
Epoch : 1,Loss: 2.242, Acc: 0.344
Epoch : 1,Loss: 2.218, Acc: 0.594
Epoch : 1,Loss: 2.228, Acc: 0.406
Epoch : 1,Loss: 2.227, Acc: 0.5
Epoch : 1,Loss: 2.213, Acc: 0.406
Epoch : 1,Loss: 2.196, Acc: 0.656
Epoch : 1,Loss: 2.22, Acc: 0.469
Epoch : 1,Loss: 2.194, Acc: 0.531
Epoch : 1,Loss: 2.187, Acc: 0.656
Epoch : 1,Loss: 2.2, Acc: 0.406
Epoch : 1,Loss: 2.209, Acc: 0.438
Epoch : 1,Loss: 2.192, Acc: 0.438
Epoch : 1,Loss: 2.171

Epoch : 1,Loss: 1.799, Acc: 0.812
Epoch : 1,Loss: 1.965, Acc: 0.562
Epoch : 1,Loss: 2.043, Acc: 0.438
Epoch : 1,Loss: 2.031, Acc: 0.438
Epoch : 1,Loss: 1.855, Acc: 0.75
Epoch : 1,Loss: 1.922, Acc: 0.594
Epoch : 1,Loss: 1.923, Acc: 0.625
Epoch : 1,Loss: 1.932, Acc: 0.594
Epoch : 1,Loss: 1.876, Acc: 0.75
Epoch : 1,Loss: 1.967, Acc: 0.469
Epoch : 1,Loss: 1.962, Acc: 0.625
Epoch : 1,Loss: 1.897, Acc: 0.625
Epoch : 1,Loss: 2.002, Acc: 0.531
Epoch : 1,Loss: 1.887, Acc: 0.75
Epoch : 1,Loss: 1.988, Acc: 0.531
Epoch : 1,Loss: 1.914, Acc: 0.688
Epoch : 1,Loss: 1.885, Acc: 0.75
Epoch : 1,Loss: 1.887, Acc: 0.656
Epoch : 1,Loss: 1.906, Acc: 0.625
Epoch : 1,Loss: 1.941, Acc: 0.688
Epoch : 1,Loss: 1.967, Acc: 0.625
Epoch : 1,Loss: 1.915, Acc: 0.656
Epoch : 1,Loss: 2.033, Acc: 0.5
Epoch : 1,Loss: 1.907, Acc: 0.656
Epoch : 1,Loss: 1.946, Acc: 0.625
Epoch : 1,Loss: 1.873, Acc: 0.75
Epoch : 1,Loss: 1.915, Acc: 0.688
Epoch : 1,Loss: 1.911, Acc: 0.719
Epoch : 1,Loss: 1.885, Acc: 0.688
Epoch : 1,Loss: 1.897

Epoch : 1,Loss: 1.761, Acc: 0.812
Epoch : 1,Loss: 1.865, Acc: 0.688
Epoch : 1,Loss: 1.855, Acc: 0.719
Epoch : 1,Loss: 1.907, Acc: 0.625
Epoch : 1,Loss: 1.833, Acc: 0.719
Epoch : 1,Loss: 1.863, Acc: 0.688
Epoch : 1,Loss: 1.907, Acc: 0.594
Epoch : 1,Loss: 1.804, Acc: 0.75
Epoch : 1,Loss: 1.894, Acc: 0.594
Epoch : 1,Loss: 1.791, Acc: 0.75
Epoch : 1,Loss: 1.855, Acc: 0.656
Epoch : 1,Loss: 1.852, Acc: 0.656
Epoch : 1,Loss: 1.842, Acc: 0.781
Epoch : 1,Loss: 1.763, Acc: 0.875
Epoch : 1,Loss: 1.875, Acc: 0.656
Epoch : 1,Loss: 1.834, Acc: 0.719
Epoch : 1,Loss: 1.874, Acc: 0.719
Epoch : 1,Loss: 1.806, Acc: 0.781
Epoch : 1,Loss: 1.773, Acc: 0.719
Epoch : 1,Loss: 1.747, Acc: 0.906
Epoch : 1,Loss: 1.89, Acc: 0.625
Epoch : 1,Loss: 1.904, Acc: 0.625
Epoch : 1,Loss: 1.811, Acc: 0.719
Epoch : 1,Loss: 1.743, Acc: 0.812
Epoch : 1,Loss: 1.864, Acc: 0.625
Epoch : 1,Loss: 1.865, Acc: 0.719
Epoch : 1,Loss: 1.746, Acc: 0.875
Epoch : 1,Loss: 1.915, Acc: 0.594
Epoch : 1,Loss: 1.743, Acc: 0.844
Epoch : 1,Loss: 1

Epoch : 1,Loss: 1.815, Acc: 0.719
Epoch : 1,Loss: 1.853, Acc: 0.688
Epoch : 1,Loss: 1.844, Acc: 0.688
Epoch : 1,Loss: 1.749, Acc: 0.781
Epoch : 1,Loss: 1.827, Acc: 0.625
Epoch : 1,Loss: 1.76, Acc: 0.75
Epoch : 1,Loss: 1.825, Acc: 0.75
Epoch : 1,Loss: 1.788, Acc: 0.812
Epoch : 1,Loss: 1.769, Acc: 0.75
Epoch : 1,Loss: 1.899, Acc: 0.625
Epoch : 1,Loss: 1.947, Acc: 0.562
Epoch : 1,Loss: 1.816, Acc: 0.688
Epoch : 1,Loss: 1.819, Acc: 0.688
Epoch : 1,Loss: 1.742, Acc: 0.781
Epoch : 1,Loss: 1.79, Acc: 0.781
Epoch : 1,Loss: 1.827, Acc: 0.688
Epoch : 1,Loss: 1.754, Acc: 0.844
Epoch : 1,Loss: 1.893, Acc: 0.594
Epoch : 1,Loss: 1.771, Acc: 0.781
Epoch : 1,Loss: 1.869, Acc: 0.656
Epoch : 1,Loss: 1.854, Acc: 0.594
Epoch : 1,Loss: 1.814, Acc: 0.688
Epoch : 1,Loss: 1.837, Acc: 0.688
Epoch : 1,Loss: 1.849, Acc: 0.656
Epoch : 1,Loss: 1.784, Acc: 0.781
Epoch : 1,Loss: 1.833, Acc: 0.656
Epoch : 1,Loss: 1.966, Acc: 0.531
Epoch : 1,Loss: 1.775, Acc: 0.75
Epoch : 1,Loss: 1.83, Acc: 0.656
Epoch : 1,Loss: 1.793

Epoch : 1,Loss: 1.791, Acc: 0.719
Epoch : 1,Loss: 1.786, Acc: 0.75
Epoch : 1,Loss: 1.816, Acc: 0.625
Epoch : 1,Loss: 1.811, Acc: 0.719
Epoch : 1,Loss: 1.814, Acc: 0.781
Epoch : 1,Loss: 1.738, Acc: 0.812
Epoch : 1,Loss: 1.799, Acc: 0.688
Epoch : 1,Loss: 1.742, Acc: 0.812
Epoch : 1,Loss: 1.694, Acc: 0.844
Epoch : 1,Loss: 1.806, Acc: 0.75
Epoch : 1,Loss: 1.801, Acc: 0.688
Epoch : 1,Loss: 1.813, Acc: 0.688
Epoch : 1,Loss: 1.74, Acc: 0.812
Epoch : 1,Loss: 1.708, Acc: 0.875
Epoch : 1,Loss: 1.791, Acc: 0.719
Epoch : 1,Loss: 1.79, Acc: 0.812
Epoch : 1,Loss: 1.72, Acc: 0.812
Epoch : 1,Loss: 1.823, Acc: 0.719
Epoch : 1,Loss: 1.748, Acc: 0.781
Epoch : 1,Loss: 1.804, Acc: 0.781
Epoch : 1,Loss: 1.758, Acc: 0.812
Epoch : 1,Loss: 1.786, Acc: 0.719
Epoch : 1,Loss: 1.73, Acc: 0.812
Epoch : 1,Loss: 1.874, Acc: 0.656
Epoch : 1,Loss: 1.678, Acc: 0.812
Epoch : 1,Loss: 1.693, Acc: 0.781
Epoch : 1,Loss: 1.734, Acc: 0.812
Epoch : 1,Loss: 1.81, Acc: 0.625
Epoch : 1,Loss: 1.666, Acc: 0.844
Epoch : 1,Loss: 1.715

Epoch : 1,Loss: 1.696, Acc: 0.844
Epoch : 1,Loss: 1.866, Acc: 0.594
Epoch : 1,Loss: 1.712, Acc: 0.812
Epoch : 1,Loss: 1.79, Acc: 0.719
Epoch : 1,Loss: 1.796, Acc: 0.656
Epoch : 1,Loss: 1.72, Acc: 0.844
Epoch : 1,Loss: 1.725, Acc: 0.781
Epoch : 1,Loss: 1.697, Acc: 0.812
Epoch : 1,Loss: 1.714, Acc: 0.812
Epoch : 1,Loss: 1.726, Acc: 0.75
Epoch : 1,Loss: 1.757, Acc: 0.781
Epoch : 1,Loss: 1.834, Acc: 0.75
Epoch : 1,Loss: 1.804, Acc: 0.688
Epoch : 1,Loss: 1.777, Acc: 0.719
Epoch : 1,Loss: 1.723, Acc: 0.781
Epoch : 1,Loss: 1.819, Acc: 0.688
Epoch : 1,Loss: 1.797, Acc: 0.75
Epoch : 1,Loss: 1.734, Acc: 0.781
Epoch : 1,Loss: 1.765, Acc: 0.719
Epoch : 1,Loss: 1.84, Acc: 0.656
Epoch : 1,Loss: 1.744, Acc: 0.75
Epoch : 1,Loss: 1.696, Acc: 0.812
Epoch : 1,Loss: 1.781, Acc: 0.781
Epoch : 1,Loss: 1.794, Acc: 0.719
Epoch : 1,Loss: 1.757, Acc: 0.75
Epoch : 1,Loss: 1.882, Acc: 0.656
Epoch : 1,Loss: 1.686, Acc: 0.844
Epoch : 1,Loss: 1.699, Acc: 0.812
Epoch : 1,Loss: 1.791, Acc: 0.75
Epoch : 1,Loss: 1.803, 

Epoch : 1,Loss: 1.729, Acc: 0.812
Epoch : 1,Loss: 1.745, Acc: 0.781
Epoch : 1,Loss: 1.681, Acc: 0.812
Epoch : 1,Loss: 1.712, Acc: 0.781
Epoch : 1,Loss: 1.739, Acc: 0.781
Epoch : 1,Loss: 1.786, Acc: 0.75
Epoch : 1,Loss: 1.767, Acc: 0.719
Epoch : 1,Loss: 1.717, Acc: 0.719
Epoch : 1,Loss: 1.728, Acc: 0.781
Epoch : 1,Loss: 1.769, Acc: 0.75
Epoch : 1,Loss: 1.667, Acc: 0.812
Epoch : 1,Loss: 1.809, Acc: 0.719
Epoch : 1,Loss: 1.719, Acc: 0.812
Epoch : 1,Loss: 1.714, Acc: 0.75
Epoch : 1,Loss: 1.67, Acc: 0.844
Epoch : 1,Loss: 1.758, Acc: 0.688
Epoch : 1,Loss: 1.719, Acc: 0.812
Epoch : 1,Loss: 1.776, Acc: 0.75
Epoch : 1,Loss: 1.681, Acc: 0.812
Epoch : 1,Loss: 1.752, Acc: 0.75
Epoch : 1,Loss: 1.855, Acc: 0.625
Epoch : 1,Loss: 1.812, Acc: 0.75
Epoch : 1,Loss: 1.702, Acc: 0.812
Epoch : 1,Loss: 1.69, Acc: 0.781
Epoch : 1,Loss: 1.729, Acc: 0.781
Epoch : 1,Loss: 1.694, Acc: 0.812
Epoch : 1,Loss: 1.682, Acc: 0.812
Epoch : 1,Loss: 1.72, Acc: 0.781
Epoch : 1,Loss: 1.738, Acc: 0.781
Epoch : 1,Loss: 1.78, A

Epoch : 1,Loss: 1.677, Acc: 0.812
Epoch : 1,Loss: 1.753, Acc: 0.781
Epoch : 1,Loss: 1.866, Acc: 0.656
Epoch : 1,Loss: 1.711, Acc: 0.781
Epoch : 1,Loss: 1.678, Acc: 0.844
Epoch : 1,Loss: 1.689, Acc: 0.781
Epoch : 1,Loss: 1.647, Acc: 0.875
Epoch : 1,Loss: 1.585, Acc: 0.938
Epoch : 1,Loss: 1.741, Acc: 0.75
Epoch : 1,Loss: 1.7, Acc: 0.812
Epoch : 1,Loss: 1.818, Acc: 0.656
Epoch : 1,Loss: 1.728, Acc: 0.781
Epoch : 1,Loss: 1.755, Acc: 0.719
Epoch : 1,Loss: 1.835, Acc: 0.688
Epoch : 1,Loss: 1.834, Acc: 0.656
Epoch : 1,Loss: 1.828, Acc: 0.688
Epoch : 1,Loss: 1.731, Acc: 0.75
Epoch : 1,Loss: 1.811, Acc: 0.656
Epoch : 1,Loss: 1.906, Acc: 0.594
Epoch : 1,Loss: 1.811, Acc: 0.656
Epoch : 1,Loss: 1.76, Acc: 0.719
Epoch : 1,Loss: 1.683, Acc: 0.875
Epoch : 1,Loss: 1.722, Acc: 0.812
Epoch : 1,Loss: 1.853, Acc: 0.594
Epoch : 1,Loss: 1.674, Acc: 0.844
Epoch : 1,Loss: 1.75, Acc: 0.719
Epoch : 1,Loss: 1.667, Acc: 0.812
Epoch : 1,Loss: 1.81, Acc: 0.656
Epoch : 1,Loss: 1.726, Acc: 0.781
Epoch : 1,Loss: 1.696

In [96]:
### New images, predict using your model (Inference)
mod.eval()

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=1024, out_features=120, bias=True)
  (linear2): Linear(in_features=120, out_features=84, bias=True)
  (linear3): Linear(in_features=84, out_features=10, bias=True)
)

In [97]:
preds=[]
for batch in F_D:
    X=batch['X'].float()
    with torch.no_grad():
        p=mod(X)
        preds.append(p)

In [101]:
preds[2].shape

torch.Size([32, 10])

In [102]:
#### Some portion of my data this my model has never seen and compute model performance on that dataset

In [103]:
mnist.head(2)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
mnist.shape

(42000, 785)

In [105]:
### 42000 examples, randomly select 80% of these rows reserve them for training

In [106]:
X=mnist.drop('label',axis=1).values
y=mnist['label'].values

In [107]:
from sklearn.model_selection import train_test_split

In [108]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [109]:
X_train.shape

(33600, 784)

In [110]:
X_test.shape

(8400, 784)

In [111]:
mnist_train=Mnist(X_train,y_train)
mnist_test=Mnist(X_test,y_test)

In [112]:
mnist_train=DataLoader(mnist_train,batch_size=32)
mnist_test=DataLoader(mnist_test,batch_size=32)

In [115]:
next(iter(mnist_train))['X'].shape

torch.Size([32, 1, 28, 28])

In [120]:
def fit(mod,opt,criteria,train_data,valid_data,epochs):
    for i in range(epochs):
        for batch in train_data:
            X=batch['X'].float()
            y=batch['y'].long()
            probs=mod(X)
            loss=criteria(probs,y)
            loss.backward()
            opt.step()
            opt.zero_grad()
            acc=(probs.argmax(axis=1)==y).float().mean().item()
        print(f"Epoch : {i+1},Loss: {round(loss.item(),3)}, Training Acc: {round(acc,3)}")
        mod.eval()
        for batch in valid_data:
            X=batch['X'].float()
            y=batch['y']
            with torch.no_grad():
                p=mod(X)
                acc=(p.argmax(axis=1)==y).float().mean().item()
        print(f"Epoch: {i+1}, Valid Acc: {round(acc,3)}")
            

In [121]:
mod=Lenet()
opt=optim.Adagrad(mod.parameters(),lr=0.001)
criteria=nn.CrossEntropyLoss()

In [122]:
fit(mod,opt,criteria,mnist_train,mnist_test,epochs=1)

Epoch : 1,Loss: 1.644, Training Acc: 0.969
Epoch: 1, Valid Acc: 0.938


In [123]:
#### 
def O(I,K,P,S):
    return ((I-K+2*P)/S) +1

In [125]:
O(32,5,0,1) ##k=5,p=0,s=1

28.0

In [126]:
O(28,4,1,2) ##k=4,p=1,s=2

14.0

In [127]:
O(14,5,0,1) ##k=5,p=0,s=1

10.0

In [128]:
O(10,4,1,2) ##k=4,p=1,s=2

5.0

In [131]:
class Lenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,6,5,1,0)
        self.pool1=nn.MaxPool2d(4,2,1)
        self.conv2=nn.Conv2d(6,16,5,1,0)
        self.pool2=nn.MaxPool2d(4,2,1)
        self.linear1=nn.Linear(16*5*5,120)
        self.linear2=nn.Linear(120,84)
        self.linear3=nn.Linear(84,10)
    def forward(self,X): ##b,1,32,32
        X=self.conv1(X) ##b,6,28,28
        X=torch.relu(X)
        X=self.pool1(X)
        X=self.conv2(X)## b,16,10,10
        X=torch.relu(X)
        X=self.pool2(X)## b,16,5,5
        X=X.view(-1,16*5*5)
        X=self.linear1(X)
        X=torch.tanh(X)
        X=self.linear2(X)
        X=torch.tanh(X)
        X=self.linear3(X)
        X=torch.softmax(X,axis=1)
        return X       

In [132]:
test_batch=torch.randn(10,1,32,32)

In [133]:
mod1=Lenet()

In [135]:
mod1(test_batch).shape

torch.Size([10, 10])

In [145]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block=nn.Sequential(nn.Conv2d(1,6,5,1,0),
                                     nn.ReLU(),
                                     nn.MaxPool2d(4,2,1),
                                     nn.Conv2d(6,16,5,1,0),
                                     nn.ReLU(),
                                     nn.MaxPool2d(4,2,1))
        self.classifier=nn.Sequential(nn.Linear(16*5*5,120),
                                     nn.Tanh(),
                                     nn.Linear(120,84),
                                     nn.Tanh(),
                                     nn.Linear(84,10))
    def forward(self,X):
        X=self.conv_block(X)
        X=X.view(-1,16*5*5)
        X=self.classifier(X)
        X=torch.softmax(X,axis=1)
        return X

In [146]:
mod2=LeNet()

In [147]:
mod2(test_batch)

tensor([[0.0985, 0.0967, 0.0810, 0.1099, 0.1077, 0.1054, 0.1049, 0.0922, 0.1009,
         0.1030],
        [0.0944, 0.0946, 0.0794, 0.1092, 0.1096, 0.1108, 0.1022, 0.0957, 0.1020,
         0.1022],
        [0.0968, 0.0964, 0.0815, 0.1098, 0.1073, 0.1085, 0.1026, 0.0926, 0.1011,
         0.1033],
        [0.0989, 0.0969, 0.0815, 0.1104, 0.1039, 0.1062, 0.1043, 0.0930, 0.0998,
         0.1052],
        [0.0960, 0.0963, 0.0827, 0.1104, 0.1090, 0.1090, 0.1012, 0.0927, 0.0993,
         0.1035],
        [0.0959, 0.0978, 0.0811, 0.1099, 0.1081, 0.1074, 0.1048, 0.0918, 0.0996,
         0.1036],
        [0.0996, 0.0961, 0.0809, 0.1112, 0.1103, 0.1059, 0.1037, 0.0914, 0.0978,
         0.1031],
        [0.0958, 0.0959, 0.0817, 0.1149, 0.1042, 0.1098, 0.1011, 0.0931, 0.1010,
         0.1025],
        [0.0991, 0.0968, 0.0830, 0.1061, 0.1056, 0.1111, 0.1053, 0.0911, 0.0989,
         0.1031],
        [0.0942, 0.0974, 0.0815, 0.1071, 0.1078, 0.1086, 0.1035, 0.0913, 0.1013,
         0.1075]], grad_fn=<

In [148]:
class VGG16(nn.Module):
    def __init__(self,num_classes,h,w):
        super().__init__()
        self.block1=nn.Sequential(nn.Conv2d(3,64,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(64,64,3,1,1),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        self.block2=nn.Sequential(nn.Conv2d(64,128,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(128,128,3,1,1),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        self.block3=nn.Sequential(nn.Conv2d(128,256,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(256,256,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(256,256,3,1,1),
                                 nn.ReLU(), 
                                 nn.MaxPool2d(2,2))
        self.block4=nn.Sequential(nn.Conv2d(256,512,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(), 
                                 nn.MaxPool2d(2,2))
        self.block5=nn.Sequential(nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(), 
                                 nn.MaxPool2d(2,2))
        self.linear=nn.Sequential(nn.Linear(7*7*512,4096),
                                 nn.ReLU(),
                                 nn.Linear(4096,4096),
                                 nn.ReLU(),
                                 nn.Linear(4096,num_classes))
    def forward(self,X):
        X=self.block1(X)
        X=self.block2(X)
        X=self.block3(X)
        X=self.block4(X)
        X=self.block5(X) ## b,c,h,w
        X=X.view(-1,X.shape[1]*X.shape[2]*X.shape[3])
        X=self.linear(X)
        X=torch.softmax(X,axis=1)
        return X

In [149]:
mod3=VGG16(5)

In [150]:
test_batch=torch.randn(7,3,224,224)

In [152]:
mod3(test_batch).shape

torch.Size([7, 5])

In [153]:
class VGG16(nn.Module):
    def __init__(self,num_classes,h,w):
        super().__init__()
        self.h=h
        self.w=w
        self.block1=nn.Sequential(nn.Conv2d(3,64,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(64,64,3,1,1),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        self.block2=nn.Sequential(nn.Conv2d(64,128,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(128,128,3,1,1),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2,2))
        self.block3=nn.Sequential(nn.Conv2d(128,256,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(256,256,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(256,256,3,1,1),
                                 nn.ReLU(), 
                                 nn.MaxPool2d(2,2))
        self.block4=nn.Sequential(nn.Conv2d(256,512,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(), 
                                 nn.MaxPool2d(2,2))
        self.block5=nn.Sequential(nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(),
                                 nn.Conv2d(512,512,3,1,1),
                                 nn.ReLU(), 
                                 nn.MaxPool2d(2,2))
        self.linear=nn.Sequential(nn.Linear(self._feature_dim(),4096),
                                 nn.ReLU(),
                                 nn.Linear(4096,4096),
                                 nn.ReLU(),
                                 nn.Linear(4096,num_classes))
    def _feature_dim(self):
        x=torch.randn(1,3,self.h,self.w)
        x=self.block1(x)
        x=self.block2(x)
        x=self.block3(x)
        x=self.block4(x)
        x=self.block5(x)
        c=x.shape[1]
        h=x.shape[2]
        w=x.shape[3]
        return c*h*w
    def forward(self,X):
        X=self.block1(X)
        X=self.block2(X)
        X=self.block3(X)
        X=self.block4(X)
        X=self.block5(X) ## b,c,h,w
        X=X.view(-1,X.shape[1]*X.shape[2]*X.shape[3])
        X=self.linear(X)
        X=torch.softmax(X,axis=1)
        return X

In [154]:
mod3=VGG16(5,256,256)

In [155]:
test_batch=torch.randn(7,3,256,256)

In [156]:
mod3(test_batch).shape

torch.Size([7, 5])